In [3]:
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
import tensorflow.python.keras
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.utils.np_utils import to_categorical
from tensorflow.python.keras.callbacks import LearningRateScheduler, ModelCheckpoint
import random
from PIL import Image
from PIL import ImageOps
import os
import pandas as pd
import cv2

# pd.options.mode.chained_assignment = None
%matplotlib inline
np.random.seed(1234)
print('import successfully completed')

import successfully completed


In [4]:
testdata = pd.read_csv('Test.csv')
traindata = pd.read_csv('Train.csv')

fTrain, trainLabels = traindata['Path'], traindata['ClassId']
fTest, testLabels = testdata['Path'], testdata['ClassId']

print(fTrain.shape)
print(fTest.shape)
print(trainLabels.shape)
print(testLabels.shape)

(39209,)
(12630,)
(39209,)
(12630,)


In [5]:
def init(img):
    img = cv2.resize(img, (32, 32))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.equalizeHist(img)
    img = img/255
    return img
print('methods declaration section')

methods declaration section


In [6]:
data=[]

for i in range(fTrain.size) :
    image = cv2.imread( fTrain[i])
    inited = init(image)   
    data.append(np.array(inited))

Cells=np.array(data)
trainLabels=np.array(trainLabels)

s=np.arange(Cells.shape[0])
np.random.shuffle(s)
Cells=Cells[s]
trainLabels=trainLabels[s]
print(Cells.shape)
print(trainLabels.shape)

error: OpenCV(3.4.2) /opt/concourse/worker/volumes/live/9523d527-1b9e-48e0-7ed0-a36adde286f0/volume/opencv-suite_1535558719691/work/modules/imgproc/src/resize.cpp:4044: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
(X_train,X_val)=Cells[(int)(0.2*len(trainLabels)):],Cells[:(int)(0.2*len(trainLabels))]
(y_train,y_val)=trainLabels[(int)(0.2*len(trainLabels)):],trainLabels[:(int)(0.2*len(trainLabels))]
print(X_train.shape)
print(X_val.shape)
y_train = to_categorical(y_train, 43)
y_val = to_categorical(y_val, 43)
print(y_train.shape)
print(y_val.shape)

In [ ]:
assert(X_train.shape[0] == y_train.shape[0]), "The number of images is not equal to the number of labels."
assert(X_train[0].shape == (32,32)), "The dimensions of the images are not 32 x 32."
assert(X_val.shape[0] == y_val.shape[0]), "The number of images is not equal to the number of labels."
assert(X_val[0].shape == (32,32)), "The dimensions of the images are not 32 x 32."
print('assertion true') 

In [ ]:
plt.imshow(X_train [9009], cmap='gray')
plt.axis('off')

In [ ]:
plt.imshow(X_val[100], cmap='gray')
plt.axis('off')

In [ ]:
def createModel():
    model = Sequential()
    model.add(Conv2D(60, (5, 5), input_shape=(32, 32, 1), activation='relu'))
    model.add(Conv2D(30, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    model.add(Conv2D(30, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(43, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
print('model creation method')

In [ ]:
model = createModel()
print(model.summary())

In [ ]:
X_train=X_train.reshape(31368,32, 32,1)
X_val=X_val.reshape(7841,32, 32,1)
print(X_train.shape)
print(X_train.shape)

In [ ]:
epochs = 20
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(X_val, y_val))
model.save('myModel.h5')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.xlabel('epoch')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training','test'])
plt.title('Accuracy')
plt.xlabel('epoch')

In [ ]:
print(fTest.shape)
print(testLabels.shape)

In [ ]:
testData=[]
for i in range(fTest.size) :
    image = cv2.imread( fTest[i])
    inited = init(image)   
    testData.append(np.array(inited))
testData = np.array(testData)
testLabels = np.array(testLabels)
print(testData.shape)
print(testLabels.shape)

In [ ]:
testData = testData.reshape(12630,32, 32,1)
testLabels = to_categorical(testLabels, 43)
print(testData.shape)
print(testLabels.shape)

In [ ]:
assert(testData.shape[0] == testLabels.shape[0]), "The number of images is not equal to the number of labels."
assert(testData[0].shape == (32,32, 1)), "The dimensions of the images are not 32 x 32."
print('assertion true') 

In [ ]:
score = model.evaluate(testData, testLabels, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])
